In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [6]:
data18 = data.loc[data['Year']== '2018']
data18 = data18.loc[data18['Genre1']== 'Rap']
data18

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN
12,13,2 Minute Drills EP,"ALLBLACK,Kenny Beats",not everyone can discover new ways to say they...,"Play Runners Association ,D.O.T.S.,EMPIRE",7.4,November,19,2018,Rap,NaN,NaN,NaN
17,18,Creed II: The Album,Mike WiLL Made-It,"the standout scene in , isn’t a fight. it’s ad...","Eardruma,Interscope",5.7,November,28,2018,Rap,NaN,NaN,NaN
79,80,G I R L C O D E,City Girls,"’ , replicates the winning formula of their mi...","Quality Control,Motown,Capitol",7.0,November,28,2018,Rap,NaN,NaN,NaN
81,82,Some Rap Songs,Earl Sweatshirt,"if the title of ,’s long-awaited third studio ...","Tan Cressida,Columbia",8.8,November,30,2018,Rap,NaN,NaN,NaN
84,85,Magnetic Memory,Hprizm,"the opening gambit of ,’s , is a familiar one....",Don Giovanni,7.7,November,30,2018,Rap,NaN,NaN,NaN
90,91,WRLD on Drugs,"Future,Juice WRLD",plays a narcissist who wants nothing more tha...,Epic Records,6.3,October,25,2018,Rap,NaN,NaN,NaN
94,95,Nuthin’ 2 Prove,Lil Yachty,"is why , should not be spitting bars. the bes...","Quality Control,Capitol,Motown",6.1,October,24,2018,Rap,NaN,NaN,NaN
102,103,Heavy Down Pour EP,Zubin,zubin doesn’t really sound like anyone else in...,Misery Club,6.9,December,6,2018,Rap,NaN,NaN,NaN


In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data18['content']))

print(data_words[:1])

[['it', 'difference', 'between', 'beatmaker', 'and', 'producer', 'producer', 'said', 'in', 'recent', 'interview', 'with', 'not', 'calling', 'you', 'to', 'just', 'jump', 'on', 'something', 'no', 'we', 're', 'producing', 'here', 'this', 'is', 'the', 'concept', 'mma', 'put', 'you', 'on', 'this', 'put', 'this', 'here', 'do', 'that', 'there', 'say', 'this', 'line', 'instead', 'of', 'that', 'line', 'it', 'levels', 'to', 'it', 'the', 'implication', 'is', 'that', 'producer', 'is', 'hands', 'on', 'where', 'beatmaker', 'is', 'not', 'and', 'has', 'greater', 'impact', 'on', 'the', 'song', 'direction', 'without', 'necessarily', 'playing', 'on', 'it', 'in', 'an', 'almost', 'advisory', 'role', 'that', 'seems', 'to', 'be', 'an', 'important', 'caveat', 'for', 'his', 'new', 'album', 'which', 'bills', 'him', 'as', 'lead', 'artist', 'despite', 'the', 'fact', 'that', 'he', 'rarely', 'raps', 'and', 'made', 'only', 'half', 'the', 'beats', 'was', 'culled', 'from', 'massive', 'backlog', 'and', 'whittled', 'int

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['it', 'difference', 'between', 'beatmaker', 'and', 'producer', 'producer', 'said', 'in', 'recent', 'interview', 'with', 'not', 'calling', 'you', 'to', 'just', 'jump', 'on', 'something', 'no', 'we', 're', 'producing', 'here', 'this', 'is', 'the', 'concept', 'mma', 'put', 'you', 'on', 'this', 'put', 'this', 'here', 'do', 'that', 'there', 'say', 'this', 'line', 'instead', 'of', 'that', 'line', 'it', 'levels', 'to', 'it', 'the', 'implication', 'is', 'that', 'producer', 'is', 'hands', 'on', 'where', 'beatmaker', 'is', 'not', 'and', 'has', 'greater', 'impact', 'on', 'the', 'song', 'direction', 'without', 'necessarily', 'playing', 'on', 'it', 'in', 'an', 'almost', 'advisory', 'role', 'that', 'seems', 'to', 'be', 'an', 'important', 'caveat', 'for', 'his', 'new', 'album', 'which', 'bills', 'him', 'as', 'lead', 'artist', 'despite', 'the', 'fact', 'that', 'he', 'rarely', 'raps', 'and', 'made', 'only', 'half', 'the', 'beats', 'was', 'culled', 'from', 'massive', 'backlog', 'and', 'whittled', 'into

In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['difference', 'beatmaker', 'producer', 'producer', 'say', 'recent', 'interview', 'call', 'jump', 'something', 'produce', 'concept', 'mma', 'put', 'put', 'say', 'line', 'instead', 'line', 'level', 'implication', 'producer', 'hand', 'beatmaker', 'great', 'impact', 'song', 'direction', 'necessarily', 'play', 'almost', 'advisory', 'role', 'seem', 'important', 'caveat', 'new', 'album', 'bill', 'lead', 'artist', 'fact', 'rarely', 'rap', 'make', 'half', 'beat', 'cull', 'massive', 'backlog', 'whittle', 'curat', 'rap', 'relic', 'project', 'album', 'energy', 'album', 'motivation', 'acoustic', 'album', 'vibe', 'global', 'album', 'feature', 'international', 'artist', 'early', 'version', 'album', 'feature', 'change', 'direction', 'lead', 'pick', 'song', 'make', 'call', 'east_coast', 'worth', 'east_coast', 'even', 'close', 'album', 'helm', 'titan', 'peak', 'power', 'deliver', 'colossal', 'stake', 'far', 'less', 'vision', 'less', 'focus', 'stake', 'whatsoever', 'essential', 'transcendent', 'album',

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 3), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 2), (65, 1), (66, 2), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 2)]]


In [14]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [15]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.099*"ep" + 0.045*"pattern" + 0.033*"bay_area" + 0.021*"wit" + '
  '0.016*"boast" + 0.015*"heavy" + 0.015*"shoot" + 0.014*"closer" + '
  '0.014*"underground" + 0.014*"hip_hop"'),
 (1,
  '0.038*"future" + 0.024*"gon" + 0.016*"matter" + 0.015*"buzz" + '
  '0.014*"stand" + 0.014*"dream" + 0.013*"barely" + 0.012*"energy" + '
  '0.011*"triumph" + 0.011*"party"'),
 (2,
  '0.001*"check" + 0.001*"switch" + 0.001*"king" + 0.001*"gold" + '
  '0.001*"center" + 0.001*"event" + 0.001*"group" + 0.001*"vulnerable" + '
  '0.001*"red" + 0.001*"refer"'),
 (3,
  '0.001*"throw" + 0.001*"wait" + 0.001*"dance" + 0.001*"flash" + '
  '0.001*"lyrical" + 0.001*"short" + 0.001*"exactly" + 0.001*"root" + '
  '0.001*"king" + 0.001*"anyone"'),
 (4,
  '0.010*"genre" + 0.009*"speak" + 0.009*"shift" + 0.008*"base" + '
  '0.008*"talent" + 0.008*"gold" + 0.008*"post" + 0.008*"stretch" + '
  '0.008*"rock" + 0.007*"lp"'),
 (5,
  '0.284*"thug" + 0.040*"fully" + 0.016*"dynamic" + 0.012*"continue" + '
  '0.004*"cold

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.88507632400448

Coherence Score:  0.2287111154721754


In [17]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.021775  0.020810       1        1  38.597794
9     -0.010695 -0.063840       2        1  18.461292
6      0.017564  0.038670       3        1  17.591175
4      0.026273  0.040917       4        1  14.777526
7     -0.015338 -0.276400       5        1   5.346272
1     -0.258717  0.102808       6        1   3.468719
0      0.193898  0.097910       7        1   1.607714
5      0.015914  0.026797       8        1   0.115307
3      0.004663  0.006163       9        1   0.017755
2      0.004663  0.006165      10        1   0.016447, topic_info=    Category        Freq          Term       Total  loglift  logprob
648  Default   38.000000         fully   38.000000  30.0000  30.0000
450  Default   47.000000            ep   47.000000  29.0000  29.0000
412  Default   47.000000      continue   47.000000  28.0000  28.0000
713  Default   22.000000          thug   22.000000  27.0000  27.0000
162  Default   56.000000         heavy   56.000000  26.0000  26.0000
373  Default   59.000000         shoot   59.000000  25.0000  25.0000
583  Default   54.000000         crack   54.000000  24.0000  24.0000
84   Default   59.000000          baby   59.000000  23.0000  23.0000
356  Default   41.000000        future   41.000000  22.0000  22.0000
91   Default   39.000000         boast   39.000000  21.0000  21.0000
235  Default   86.000000       atlanta   86.000000  20.0000  20.0000
585  Default   52.000000         drake   52.000000  19.0000  19.0000
800  Default   23.000000       dynamic   23.000000  18.0000  18.0000
509  Default   38.000000           wit   38.000000  17.0000  17.0000
148  Default   56.000000        bounce   56.000000  16.0000  16.0000
611  Default   51.000000          fall   51.000000  15.0000  15.0000
27   Default   44.000000       hip_hop   44.000000  14.0000  14.0000
635  Default   47.000000      american   47.000000  13.0000  13.0000
164  Default   50.000000  instrumental   50.000000  12.0000  12.0000
60   Default   62.000000          rich   62.000000  11.0000  11.0000
175  Default   38.000000          spit   38.000000  10.0000  10.0000
127  Default   71.000000          rock   71.000000   9.0000   9.0000
308  Default   48.000000     emotional   48.000000   8.0000   8.0000
195  Default   37.000000     establish   37.000000   7.0000   7.0000
586  Default   51.000000          fine   51.000000   6.0000   6.0000
2    Default   47.000000        better   47.000000   5.0000   5.0000
655  Default   45.000000    soundcloud   45.000000   4.0000   4.0000
638  Default  117.000000         group  117.000000   3.0000   3.0000
18   Default   55.000000        energy   55.000000   2.0000   2.0000
438  Default   39.000000      straight   39.000000   1.0000   1.0000
..       ...         ...           ...         ...      ...      ...
639  Topic10    0.004783      internal   13.617157   0.7589  -6.6933
109  Topic10    0.004783         honor   13.660524   0.7557  -6.6933
521  Topic10    0.004783        refuse   13.663831   0.7554  -6.6933
502  Topic10    0.004784         refer   13.792474   0.7462  -6.6932
95   Topic10    0.004785         check   30.362627  -0.0427  -6.6930
180  Topic10    0.004785        switch   24.625692   0.1667  -6.6930
592  Topic10    0.004784          king   26.305136   0.1006  -6.6932
789  Topic10    0.004784          gold   37.623119  -0.2573  -6.6932
93   Topic10    0.004784        center   29.017561   0.0024  -6.6932
638  Topic10    0.004784         group  117.319328  -1.3946  -6.6932
141  Topic10    0.004784    vulnerable   15.035380   0.6599  -6.6932
540  Topic10    0.004784        public   29.453335  -0.0125  -6.6932
411  Topic10    0.004784  contemporary   34.507675  -0.1709  -6.6932
0    Topic10    0.004784           art   57.476467  -0.6811  -6.6933
545  Topic10    0.004784          seek   33.495934  -0.1412  -6.6933
231  Topic10    0.004784         whole   44.140045  -0.4171  -6.69